In [1]:
import utils, json, umap, hdbscan, umap.plot
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from config import config
from catboost import CatBoostClassifier
from ast import literal_eval
from plotly.subplots import make_subplots
from transformers import pipeline

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/chunyuko/.cache/huggingface/token
Login successful


/Users/chunyuko/Documents/anaconda3/envs/py_3_11/lib/python3.11/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


In [2]:
job_categories = pd.read_csv("data/20231217_job_category.csv")
job_infos = pd.read_csv("data/20231219_job_info.csv")

with open("data/industry_category.json", "r") as f:
    industry_category = json.load(f)
    
industry_name = {
    "健康醫療社會福利政府機構": 'Healthcare and Social Welfare',
    "專業服務": 'Professional Services',
    "廣告出版新聞媒體": 'Advertising, Publishing, and Media',
    "服務批發零售通路餐旅運輸": 'Service, Wholesale, Retail, and Transportation',
    "製造業": 'Manufacturing',
    "資通訊軟體業科技": 'Information and Communication Technology',
    "金融保險": 'Finance and Insurance'
}

In [77]:
data

,id,category,industry,experience,education,region,management
0,53ewp,Data Scientist,Manufacturing,3.0,Any,Zhongzheng,False
1,73ifb,Data Engineer,"Service, Wholesale, Retail, and Transportation",1.0,Any,Zhongshan,False
2,74fxr,Data Analyst,Finance and Insurance,NaN,Any,Zhongshan,False
3,7aflo,Data Scientist,Finance and Insurance,1.0,Any,Xinyi,False
4,7djse,Data Analyst,Professional Services,4.0,Any,Songshan,False
...,...,...,...,...,...,...,...
2034,7vm7k,Data Analyst,Information and Communication Technology,NaN,Any,Zhongzheng,False
2035,7w1fm,Data Analyst,Finance and Insurance,1.0,Any,Songshan,False
2036,8060q,Data Scientist,"Advertising, Publishing, and Media",NaN,Master,Neihu,False
2037,81ixt,Data Analyst,Manufacturing,NaN,Any,Neihu,False


In [70]:
data = job_categories.merge(job_infos, on = "id", how = "left")
data = data[data["region"].str.contains("台北市")]
data["industry"] = data["industry"].map(industry_category).map(industry_name)

location = {
    '台北市中山區': "Zhongshan",
    '台北市中正區': "Zhongzheng",
    '台北市信義區': "Xinyi",
    '台北市內湖區': "Neihu",
    '台北市北投區': "Beitou",
    '台北市南港區': "Nanngang",
    '台北市士林區': "Shilin",
    '台北市大同區': "Datong",
    '台北市大安區': "Daan",
    '台北市文山區': "Wenshan",
    '台北市松山區': "Songshan",
    '台北市萬華區': "Wanhua"
    }

data["region"] = data["region"].map(location)
data.education.fillna("Any", inplace = True)

In [107]:
region_count = data.groupby(["region"], as_index=False).agg(count=("id", "count"))
region_count["percentage"] = (region_count["count"] / region_count["count"].sum()) * 100
region_count.sort_values(by = "percentage", ascending = False, inplace = True)
region_count

,region,count,percentage
4,Neihu,482,23.920596
9,Xinyi,346,17.171216
10,Zhongshan,271,13.449132
6,Songshan,256,12.704715
1,Daan,230,11.414392
11,Zhongzheng,160,7.940447
3,Nanngang,151,7.493797
2,Datong,47,2.332506
0,Beitou,45,2.233251
5,Shilin,13,0.645161


In [104]:
region_cat_summary = data.groupby(["region", "category"], as_index=False).agg(count=("id", "count"))

In [101]:
px.icicle(region_cat_summary, path=["category", "region"], values='count', color='region')

In [110]:
industry_count = data.groupby(["industry"], as_index=False).agg(count=("id", "count"))
industry_count["percentage"] = (industry_count["count"] / industry_count["count"].sum()) * 100
industry_count.sort_values(by = "percentage", ascending = False, inplace = True)
industry_count

,industry,count,percentage
3,Information and Communication Technology,862,42.779156
1,Finance and Insurance,348,17.270471
5,Professional Services,212,10.521092
4,Manufacturing,202,10.024814
6,"Service, Wholesale, Retail, and Transportation",180,8.933002
0,"Advertising, Publishing, and Media",113,5.607940
2,Healthcare and Social Welfare,98,4.863524


In [44]:
cat_ind_summary = data.groupby(["category", "industry"], as_index=False).agg(count=("id", "count"))

In [61]:
px.scatter(cat_ind_summary,
           x="category", y="industry", 
           color="count", size="count", text="count",
           color_continuous_scale="OrRd", size_max=20).\
update_traces(textposition='top center').\
update_xaxes(title_text="").\
update_yaxes(title_text="").\
update_layout(template="plotly_white")


In [71]:
ca_id_edu_summary = data.groupby(["category", "industry", "education"], as_index=False).agg(count=("id", "count"))

In [94]:
fig = px.bar(ca_id_edu_summary, x="count", y="industry", color="education", facet_col="category")
fig.update_layout(xaxis_title="", yaxis_title="", template="plotly_white", legend=dict(x=0, y=-.2, title="Min. education required", orientation="h"))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_xaxis(lambda x: x.update(title = ""))
fig.show()